<a href="https://colab.research.google.com/github/H1357/Diploma_task/blob/master/Diploma_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Цели проекта:
###Понять как работать с аудио (копирование голоса/голос в текст / текст в голос)
# Задачи проекта: 
###1 почитать статьи на сайте архив
###2 понять как работает код для этих статей
###3 найти интересный пример и переделать его для диплома
###4 купить видеокарту и развивать проект после диплома 

In [1]:
# Clone git repo
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 2545, done.
^C


In [ ]:
cd Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
!apt-get install -qq libportaudio2
!pip install pyTelegramBotAPI
!pip install pysoundfile

In [ ]:
# Download dataset
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
!unzip pretrained.zip

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:03, 104MB/s] 
Archive:  pretrained.zip
replace encoder/saved_models/pretrained.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: encoder/saved_models/pretrained.pt  
replace synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
replace synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  y

replace synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: synthesizer/saved_models/logs-pretrained/taco_pretraine

In [ ]:
# load weights

from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


/content/Real-Time-Voice-Cloning/encoder/audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


In [ ]:
# Vocoder
def synth():
  
  text = "This is skynet. I am going to you if you don't end the diploma"
  in_fpath = path_input

  reprocessed_wav = encoder.preprocess_wav(in_fpath)
  original_wav, sampling_rate = librosa.load(in_fpath)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  print("Synthesizing new audio...")
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  
  new_output(generated_wav,synthesizer.sample_rate)
  

In [ ]:
#Path for save voice from you and from vocoder
path_output='/content/Real-Time-Voice-Cloning/temp_audio/new_output.wav'
path_input='/content/Real-Time-Voice-Cloning/temp_audio/new_input.wav'

In [ ]:
# Some additional function for telegram bot

import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import soundfile as sf  
import tempfile
import os
import numpy as np
%matplotlib inline


#save to ogg
def save_to_ogg(in_filename=None, in_bytes=None):
    with tempfile.TemporaryFile() as temp_out_file:
        temp_in_file = None
        if in_bytes:
            temp_in_file = tempfile.NamedTemporaryFile(delete=False)
            temp_in_file.write(in_bytes)
            in_filename = temp_in_file.name
            temp_in_file.close()
            print('file_name_',in_filename)
            return in_filename
        if not in_filename:
            raise Exception('Neither input file name nor input bytes is specified.')    

#transform to wav
def new_wav(path):
    data, samplerate = librosa.load(path)
    sf.write(path_input, data, samplerate)
    os.remove(path)

def new_output(data, samplerate):
    sf.write(path_output, data, samplerate)

In [ ]:
# telegram bot 

import telebot
from telebot import types

import requests


API_TOKEN = ''

bot = telebot.TeleBot(API_TOKEN)


@bot.message_handler(func=lambda message: True)
def echo_all(message):
	chat_id = message.chat.id
	bot.send_message(chat_id, 'Tell something during 9 seconds for me')

@bot.message_handler(content_types=['voice'])
def handle_(message):
	
    # lets find our audio
    file_info = bot.get_file(message.voice.file_id)
    file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path)) 
    print(file_info)
    bot.reply_to(message,'voice is taken, wait 30 sec please')
    # ogg to wav
    path_temp=save_to_ogg(in_bytes=file.content)
    new_wav(path_temp)
    synth()
    
    f = open(path_output, 'rb')
    
    msg = bot.send_voice(message.chat.id, f, None)
    os.remove(path_output)
    os.remove(path_input)
 
bot.polling()

{'file_id': 'AwACAgIAAxkBAAIBOV-UbbbZPZKLaf1YcFVPU6TwUg2QAAI-CgACBRWhSNZcDOg0MqZZGwQ', 'file_size': 18678, 'file_path': 'voice/file_21.oga'}
file_name_ /tmp/tmp7ohbkk95
Synthesizing new audio...
{| ████████████████ 76000/76800 | Batch Size: 8 | Gen Rate: 2.0kHz | }

In [ ]:
# Выводы:
###1 Надо сделать файнтюнинг на русский язык
###2 Переобучить сеть на больший датасет
###3 Попробовать поменять синтезатор и вокодер на более современные с сайта архив
###4 Попробовать распознавание и синтез эмоций на голосе